In [1]:
import random, cv2, skimage.util, numpy as np, tkinter as tk
from tkinter import *
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk, ImageFilter,ImageEnhance
import matplotlib.pyplot as plt
from skimage.feature import canny
from skimage import morphology,filters,measure,segmentation,color
from skimage.segmentation import felzenszwalb
import scipy.ndimage as nd
from skimage.color import label2rgb
from skimage.segmentation import slic
import quads
# from pyqtree import Index 
#libraries


In [4]:
class App(tk.Tk):
    def __init__(self) -> None:
        super().__init__()


class MainWin:
    def __init__(self, master) -> None:
        self.master = master
        self.master.title("Imagy")
        self.master.geometry('1080x720')

        self.canvas = Canvas(self.master)
        self.setup()
        self.l1=tk.Label(self.master,text=f'')
        self.l1.place(x=10,y=10)


        

    def setup(self):
        self.canvas.grid(row=0, column=0, sticky='nswe')
        self.canvas.update()
        self.master.rowconfigure(0, weight=1)
        self.master.columnconfigure(0, weight=1)
        self.master.bind("<Control-equal>", lambda event: self.zoom_in())
        self.master.bind("<Control-minus>", lambda event: self.zoom_out())

        self.canvas.bind('<Configure>', self.showimage)
        self.canvas.bind('<ButtonPress-1>', self.move_from)
        self.canvas.bind('<B1-Motion>', self.move_to)
        self.canvas.bind('<MouseWheel>', self.wheel)
        self.canvas.bind('<Button-5>', self.wheel)
        self.canvas.bind('<Button-4>', self.wheel)
        self.imscale = 1.0
        self.delta = 1.1
        self.image = None

        self.dir = "."


    


    def move_from(self, event):
        if (self.image != None):
            self.canvas.scan_mark(event.x, event.y)

    def move_to(self, event):
        if (self.image != None):
            self.canvas.scan_dragto(event.x, event.y, gain=1)
            self.showimage()
    
    def showimage(self, event=None):
        if (self.image != None):
            self.l1.config(text=f'Scale: {self.imscale*100:.2f}%')
            bbox1 = self.canvas.bbox(self.container)
            bbox1 = (bbox1[0] + 1, bbox1[1] + 1, bbox1[2] - 1, bbox1[3] - 1)
            bbox2 = (self.canvas.canvasx(0),
                    self.canvas.canvasy(0),
                    self.canvas.canvasx(self.canvas.winfo_width()),
                    self.canvas.canvasy(self.canvas.winfo_height()))
            bbox = [min(bbox1[0], bbox2[0]), min(bbox1[1], bbox2[1]),
                    max(bbox1[2], bbox2[2]), max(bbox1[3], bbox2[3])]
            if bbox[0] == bbox2[0] and bbox[2] == bbox2[2]:
                bbox[0] = bbox1[0]
                bbox[2] = bbox1[2]
            if bbox[1] == bbox2[1] and bbox[3] == bbox2[3]:
                bbox[1] = bbox1[1]
                bbox[3] = bbox1[3]
            self.canvas.configure(scrollregion=bbox)
            x1 = max(bbox2[0] - bbox1[0], 0)
            y1 = max(bbox2[1] - bbox1[1], 0)
            x2 = min(bbox2[2], bbox1[2]) - bbox1[0]
            y2 = min(bbox2[3], bbox1[3]) - bbox1[1]
            if int(x2 - x1) > 0 and int(y2 - y1) > 0:
                x = min(int(x2 / self.imscale), self.width)
                y = min(int(y2 / self.imscale), self.height)
                image = self.image.crop((int(x1 / self.imscale), int(y1 / self.imscale), x, y))
                imagetk = ImageTk.PhotoImage(image.resize((int(x2 - x1), int(y2 - y1))))
                imageid = self.canvas.create_image(max(bbox2[0], bbox1[0]), max(bbox2[1], bbox1[1]),
                                                anchor='nw', image=imagetk)
                self.canvas.lower(imageid)
                self.canvas.imagetk = imagetk

    def wheel(self, event=None):
        if (self.image!=None):
            x = self.canvas.canvasx(event.x)
            y = self.canvas.canvasy(event.y)
            bbox = self.canvas.bbox(self.container)
            if bbox[0] < x < bbox[2] and bbox[1] < y < bbox[3]: pass
            else: return
            scale = 1.0
            if event.num == 5 or event.delta == -120:
                i = min(self.width, self.height)
                if int(i * self.imscale) < 30: return
                self.imscale /= self.delta
                scale        /= self.delta
            if event.num == 4 or event.delta == 120:
                i = min(self.canvas.winfo_width(), self.canvas.winfo_height())
                if i < self.imscale: return
                self.imscale *= self.delta
                scale        *= self.delta
            self.canvas.scale('all', x, y, scale, scale)
            self.showimage()
    
    
    #----------------               ----------------#
    #----------------    Regular    ----------------#
    #----------------               ----------------#
    def uploadimg(self):
        self.setup()
        f_types = [('Jpg Files', '*.jpg')]
        path = filedialog.askopenfilename(filetypes=f_types)
        
        self.image = Image.open(path)
        self.path = path.split('.')
        self.width, self.height = self.image.size
        self.container = self.canvas.create_rectangle(0, 0, self.width, self.height, width=0)
        
        self.showimage()
    
    def export(self):
        if (self.image != None):
            self.image.save(f"{self.path[0]}_out.{self.path[1]}", quality=100, subsampling=0)
            messagebox.showinfo("Completed", "Image exported!")
        else:
            messagebox.showerror("File error", "No image imported")

    def openbulk(self):
        f_types = [('Jpg Files', '*.jpg')]
        
        path = filedialog.askopenfilenames(filetypes=f_types)
        bulk_images = []
        for i in path:
            bulk_images.append([Image.open(i), i])
        return bulk_images
    
    def dirconfig(self):
        self.dir = filedialog.askdirectory(initialdir=self.dir, title="Select output directory")
    
    #----------------               ----------------#
    #----------------    FIlters    ----------------#
    #----------------               ----------------#
    def regular(self, func):
        if (self.image != None):
            self.image = self.image.filter(func)
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")
    
    def bulk(self, func):        
        images = self.openbulk()
        count = 0
        for i in images:
            count += 1
            save = i[1].split("/")[-1].split('.')
            j = i[0].filter(func)
            j.save(f"{self.dir}/{save[0]}_med.{save[1]}", quality=100, subsampling=0)
        messagebox.showinfo("Completed", f"{count} Image(s) exported!")
    
    #----------------              ----------------#
    #----------------    Noises    ----------------#
    #----------------              ----------------#
    def snp(self):
        sdense = 0.2
        pdense = 0.2
        if (self.image != None):
            x = simpledialog.askfloat(title="Salt and Pepper", prompt="Enter p value",  minvalue=0, maxvalue=1)
            sdense = x
            pdense = x
            for i in range(self.image.size[0]):
                for j in range(self.image.size[1]):
                    if random.random() < sdense:
                        self.image.putpixel((i, j), (255, 255, 255) )
                    if random.random() < pdense:
                        self.image.putpixel( (i, j), (0, 0, 0) )
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")
    
    def gaussnoise(self):
        if (self.image != None):
            mean = 0
            var = 0.1
            sigma = var ** 0.5
            row, col = self.image.size
            
            gaussian = np.random.random((row, col, 1)).astype(np.float32)
            gaussian = np.concatenate((gaussian, gaussian, gaussian), axis = 2)
            gaussian_img = cv2.addWeighted(self.image, 0.75, 0.25 * gaussian, 0.25, 0)
            self.image = gaussian_img

            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")
                
    #----------------              ----------------#
    #----------------    Process    ----------------#
    #----------------              ----------------#
    def crop(self):
        if (self.image != None):
            messagebox.showinfo("Crop", "Click and drag to select a region to crop.")
            self.canvas.bind('<ButtonPress-1>', self.crop_start)
            self.canvas.bind('<B1-Motion>', self.crop_drag)
            self.canvas.bind('<ButtonRelease-1>', self.crop_end)
        else:
            messagebox.showerror("File error", "No image imported")

    def crop_start(self, event):
        self.crop_x1 = self.canvas.canvasx(event.x)
        self.crop_y1 = self.canvas.canvasy(event.y)

    def crop_drag(self, event):
        self.canvas.delete("crop_rect")
        self.crop_x2 = self.canvas.canvasx(event.x)
        self.crop_y2 = self.canvas.canvasy(event.y)
        self.canvas.create_rectangle(
            self.crop_x1,
            self.crop_y1,
            self.crop_x2,
            self.crop_y2,
            outline="red",
            tags="crop_rect"
        )

    def crop_end(self, event):
        self.canvas.delete("crop_rect")
        x1 = int(self.crop_x1 / self.imscale)
        y1 = int(self.crop_y1 / self.imscale)
        x2 = int(self.crop_x2 / self.imscale)
        y2 = int(self.crop_y2 / self.imscale)
        if x1 < 0: x1 = 0
        if y1 < 0: y1 = 0
        if x2 > self.width: x2 = self.width
        if y2 > self.height: y2 = self.height
        if x1 < x2 and y1 < y2:
            self.image = self.image.crop((x1, y1, x2, y2))
            self.width, self.height = self.image.size
            self.canvas.delete("all")
            self.container = self.canvas.create_rectangle(0, 0, self.width, self.height, width=0)
            self.showimage()

    def zoom_in(self):
        self.zoom(1.1)

    def zoom_out(self):
        self.zoom(0.9)

    def zoom(self, zoom_factor):
        if self.image is not None:
            self.imscale *= zoom_factor
            self.canvas.scale('all', 0, 0, zoom_factor, zoom_factor)
            self.showimage()
            self.l1.config(text=f'Scale: {self.imscale*100:.2f}%')
        else:
            messagebox.showerror("File error", "No image imported")

    def flip_horizontal(self):
        if (self.image != None):
            self.image = self.image.transpose(Image.FLIP_LEFT_RIGHT)
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")

    def flip_vertical(self):
        if (self.image != None):
            self.image = self.image.transpose(Image.FLIP_TOP_BOTTOM)
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")

    def show_image_properties(self):
        if self.image:
            properties_window = Toplevel(app)
            properties_window.title("Image Properties")

            # Get image dimensions
            width, height = self.image.size

            # Create labels to display image properties
            label_width = Label(properties_window, text=f"Width: {width}")
            label_height = Label(properties_window, text=f"Height: {height}")
            label_format = Label(properties_window, text=f"Format: {self.path[1].upper()}")

            # Pack labels into the window
            label_width.pack()
            label_height.pack()
            label_format.pack()
        else:
            messagebox.showerror("File error", "No image imported")

    def smooth(self):
        if (self.image != None):
            radius = simpledialog.askinteger(title="Smooth", prompt="Enter blur radius (e.g., 3):", minvalue=1)
            if radius is not None:
                self.image = self.image.filter(ImageFilter.GaussianBlur(radius=radius))
                self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")

    def binary(self):
        if self.image is not None:
            # Ask the user for a threshold value
            threshold = simpledialog.askinteger(title="Binary", prompt="Enter threshold value (0-255):", minvalue=0, maxvalue=255)
            if threshold is not None:
                # Convert the image to grayscale
                grayscale_image = self.image.convert("L")

                # Create a binary image using the threshold
                binary_image = grayscale_image.point(lambda x: 0 if x < threshold else 255, '1')

                # Update the image and display it
                self.image = binary_image
                self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")

#----------------              ----------------#
#---------------- Segmentation ----------------#
#----------------              ----------------#

    def apply_canny_detection(self):
        if self.image is not None:
            # Convert the PIL Image to OpenCV format
            image = np.array(self.image)

            # Apply Canny edge detection
            edges = cv2.Canny(image, 100, 200)

            # Convert the result back to PIL Image format
            self.image = Image.fromarray(edges)

            # Show the processed image
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")    


    def segment_edges_sobel(self):
        if self.image is not None:
            # Convert the PIL Image to OpenCV format
            image = np.array(self.image)

            # Apply Sobel operator for edge detection
            # The arguments for the Sobel function are:
            # - The source image.
            # - The order of the derivative in the x direction.
            # - The order of the derivative in the y direction.
            # - The aperture size for the Sobel kernel (it must be 1, 3, 5, or 7).
            # - The scale factor for the computed derivative values.
            # - The data type of the computed derivative values.
            # - The border type.
            # - The constant border value.
            edges = cv2.Sobel(image, cv2.CV_64F, 1, 1)

            # Convert the result back to PIL Image format
            self.image = Image.fromarray(edges)

            # Show the processed image
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")
            



    def felzenszwalb_segmentation(self):
        if self.image is not None:
            # Convert the PIL Image to OpenCV format
            image = np.array(self.image)

            # Ask the user for the scale parameter for the Felzenszwalb algorithm
            scale = simpledialog.askinteger("Felzenszwalb Segmentation", "Enter the scale parameter for the Felzenszwalb algorithm(1-1000):")
            if scale is None:
                scale = 100
                
            # Apply Felzenszwalb segmentation
            segments = felzenszwalb(image, scale=scale)

            # Convert the result back to PIL Image format
            self.image = Image.fromarray(label2rgb(segments, image, kind='avg'))

            # Show the processed image
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")


    def gaussian_adaptive_thresholding(self):
        if self.image is not None:
            # Convert the image to grayscale
            grayscale_image = self.image.convert("L")

            # Ask the user for a block size
            block_size = simpledialog.askinteger(title="Gaussian Adaptive Thresholding", prompt="Enter block size (e.g., 15):", minvalue=1)
            if block_size is None:
                block_size = 15

            # Apply Gaussian Adaptive Thresholding
            binary_image = grayscale_image.point(lambda x: 0 if x < 127 else 255, '1')

            # Update the image and display it
            self.image = binary_image
            self.showimage()
        else:
            messagebox.showerror("File error", "No image imported")



#----------------              ----------------#
#----------------    Windowsmenu    ----------------#
#----------------              ----------------#

class WMenu(tk.Menu):
    def __init__(self, master) -> None:
        super(WMenu, self).__init__()

        self.add_cascade(label = 'File', menu = self.setupfile())
        self.add_cascade(label = 'Edit', menu = self.setupedit())
        self.add_cascade(label = 'Bulk', menu = self.setupbulk())
        self.add_cascade(label = 'Process', menu = self.setupprocess())
        self.add_cascade(label = 'Segmentation', menu = self.setupsegmentation())


    # TOP LEVEL MENUS
    def setupfile(self):
        menu = Menu(self, tearoff=0)
        menu.add_command(label = "Import", command=mwin.uploadimg)
        menu.add_command(label = "Export", command=mwin.export)
        menu.add_separator()
        menu.add_command(label = "Exit", command = app.destroy)
        return menu
    
    def setupedit(self):
        menu = Menu(self, tearoff=0)
        menu.add_cascade(label = "Noise", menu = self.setupnoise())
        menu.add_cascade(label = "Filter", menu = self.setupfilter())
        return menu

    def setupbulk(self):
        menu = Menu(self, tearoff=0)
        menu.add_command(label = "Average Filter", command=lambda: mwin.bulk(ImageFilter.BoxBlur(radius = 3)))
        menu.add_command(label = "Median Filter", command=lambda: mwin.bulk(ImageFilter.MedianFilter(size = 3)))
        menu.add_command(label = "Min Filter", command=lambda: mwin.bulk(ImageFilter.MinFilter(size = 3)))
        menu.add_command(label = "Max Filter", command=lambda: mwin.bulk(ImageFilter.MaxFilter(size = 3)))
        menu.add_command(label = "Mode Filter", command=lambda: mwin.bulk(ImageFilter.ModeFilter(size = 3)))
        menu.add_command(label = "Gaussian Filter", command=lambda: mwin.bulk(ImageFilter.GaussianBlur(radius = 3)))
        menu.add_separator()
        menu.add_command(label = "Config", command=mwin.dirconfig)
        return menu
    
    #working on
    def setupprocess(self):
        menu=Menu(self,tearoff=0)
        menu.add_command(label = "Crop",  command=lambda: mwin.crop())
        menu.add_command(label="Zoom In", command=lambda: mwin.zoom_in(), accelerator="Ctrl + +")
        menu.add_command(label="Zoom Out", command=lambda: mwin.zoom_out(), accelerator="Ctrl + -")
        self.master.bind("<Control-equal>", lambda event: mwin.zoom_in())
        self.master.bind("<Control-minus>", lambda event: mwin.zoom_out())
        menu.add_cascade(label = "Tranform", menu = self.setuptransform())
        menu.add_command(label = "Smooth",  command=lambda:  mwin.smooth())
        menu.add_command(label = "Binary",  command=lambda:  mwin.binary())
        menu.add_separator()
        menu.add_cascade(label = "Properties", command=lambda: mwin.show_image_properties())
        return menu
    
    def setupsegmentation(self):
        menu=Menu(self,tearoff=0)
        menu.add_cascade(label = "Edge Based", menu = self.edgebasedsegmentation())

        return menu
    
   


    # 2nd LEVEL MENUS
    def setupnoise(self):
        menu = Menu(self, tearoff=0)
        menu.add_command(label = "Salt and Pepper", command=mwin.snp)
        menu.add_command(label = "Gaussian", command=mwin.gaussnoise)
        return menu

    def setupfilter(self):
        menu = Menu(self, tearoff=0)
        menu.add_command(label = "Average", command=lambda: mwin.regular(ImageFilter.BoxBlur(radius = 3)))
        menu.add_command(label = "Median", command=lambda: mwin.regular(ImageFilter.MedianFilter(size = 3)))
        menu.add_command(label = "Min", command=lambda: mwin.regular(ImageFilter.MinFilter(size = 3)))
        menu.add_command(label = "Max", command=lambda: mwin.regular(ImageFilter.MaxFilter(size = 3)))
        menu.add_command(label = "Mode", command=lambda: mwin.regular(ImageFilter.ModeFilter(size = 3)))
        menu.add_command(label = "Gaussian", command=lambda: mwin.regular(ImageFilter.GaussianBlur(radius = 3)))
        return menu
    

    def setuptransform(self):
        menu = Menu(self, tearoff=0)
        menu.add_command(label = "Flip Horizontally", command=lambda: mwin.flip_horizontal())
        menu.add_command(label = "Flip Vertically", command=lambda: mwin.flip_vertical())
        return menu
    
    def edgebasedsegmentation(self):
        menu = Menu(self, tearoff=0)
        menu.add_command(label = "Canny", command=mwin.apply_canny_detection)
        menu.add_command(label = "Sobel", command=mwin.segment_edges_sobel)
        menu.add_command(label = "Felzenswalb",  command=lambda: mwin.felzenszwalb_segmentation())
        menu.add_command(label = "Guassian Adaptive",  command=lambda: mwin.gaussian_adaptive_thresholding())
        
        return menu
    

app = App()

mwin = MainWin(app)
menu = WMenu(app)
app.config(menu = menu)

app.mainloop()
